In [3]:
import pandas as pd
import json
import os


projects = ["piggymetrics", "litemall", "mall", "mall-swarm", "jetlinks-community", "music-website", "pig", "spring-boot-admin", "Spring-Cloud-Platform", "apollo"]
index_name = "without"

for name in projects:

    baseline_file = f"../data/evaluation/data/{name}_dependencies.csv"
    gpt40_file = f"../data/evaluation/results/gpt4o/{name}_dependencies_{index_name}.json"
    gpt35_file = f"../data/evaluation/results/gpt3.5/{name}_dependencies_{index_name}.json"
    llama38b_file = f"../data/evaluation/results/llama3:8b/{name}_dependencies_{index_name}.json"
    llama370b_file = f"../data/evaluation/results/llama3:70b/{name}_dependencies_{index_name}.json"

    
    df = pd.read_csv(baseline_file)

    # gtp-4o
    with open(gpt40_file, "r", encoding="utf-8") as src:
        isDependency = []
        data = json.load(src)
        for x in data:
            try:
                response = json.loads(x["response"])
                isDependency.append(response["isDependency"])
            except json.JSONDecodeError:
                isDependency.append(None)

        df[f"gpt-4o_{index_name}"] = isDependency
    
    # gpt-3.5-turbo
    with open(gpt35_file, "r", encoding="utf-8") as src:
        isDependency = []
        data = json.load(src)
        for x in data:
            try:
                response = json.loads(x["response"])
                isDependency.append(response["isDependency"])
            except json.JSONDecodeError:
                print("Error")
                isDependency.append(None)
    
    df[f"gpt-3.5-turbo_{index_name}"] = isDependency
    
    # llama3:8b
    with open(llama38b_file, "r", encoding="utf-8") as src:
        isDependency = []
        data = json.load(src)
        for x in data:
            try:
                response = json.loads(x["response"])
                isDependency.append(response["isDependency"])
            except (json.JSONDecodeError, KeyError):
                isDependency.append(None)
    
        df[f"llama3:8b_{index_name}"] = isDependency

    
    # llama3:70b
    with open(llama370b_file, "r", encoding="utf-8") as src:
        isDependency = []
        data = json.load(src)
        for x in data:
            try:
                response = json.loads(x["response"])
                isDependency.append(response["isDependency"])
            except (json.JSONDecodeError, KeyError):
                isDependency.append(None)
    
        df[f"llama3:70b_{index_name}"] = isDependency

    df.to_csv(baseline_file, index=False)

In [2]:
import pandas as pd
import glob

dir = "../data/evaluation/data/"

dfs = []

for file_path in glob.glob("../data/evaluation/data/**"):
    df = pd.read_csv(file_path)
    dfs.append(df)


df_all = pd.concat(dfs)

df_all.to_csv("../data/evaluation/all_dependencies.csv", index=False)

In [12]:
import json 

gpt40_file = f"../data/evaluation/config2/all_dependencies_all_gpt-4o-2024-05-13.json"
gpt35_file = f"../data/evaluation/config2/all_dependencies_all_gpt-3.5-turbo-0125.json"
llama38b_file = f"../data/evaluation/config2/all_dependencies_all_llama3:8b.json"
llama370b_file = f"../data/evaluation/config2/all_dependencies_all_llama3:70b.json"

def read_json_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)
    
gpt40_data = read_json_file(gpt40_file)
gpt35_data = read_json_file(gpt35_file)
llama38b_data = read_json_file(llama38b_file)
llama370b_data = read_json_file(llama370b_file)

responses = []
for a, b, c, d in zip(gpt40_data, gpt35_data, llama38b_data, llama370b_data):
    response = {
        "gpt-4o-2024-05-13": a["response"],
        "gpt-3.5-turbo-0125": b["response"],
        "llama3:8b": c["response"],
        "llama3:70b": d["response"]
    }
    responses.append(response)


data_file = f"../data/evaluation/config2/all_dependencies_all.json"
data = read_json_file(data_file)

for x, y in zip(data, responses):
    x["responses"] = y


with open(f"../data/evaluation/config2/all_dependencies_all_results.json", "w", encoding="utf-8") as dest:
    json.dump(data, dest, indent=2)